In [2]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
sys.path.append('../../notebooks/entregable/scripts')
import dataset
import preprocesamiento
import target
import feature_engineering
importlib.reload(dataset)
importlib.reload(preprocesamiento)
importlib.reload(target)
importlib.reload(feature_engineering)

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


<module 'feature_engineering' from 'c:\\Users\\Usuario\\Documents\\Universidad\\austral\\2025\\Lab3\\Lab3-MCD\\notebooks\\model_prophet\\../../notebooks/entregable/scripts\\feature_engineering.py'>

In [3]:
df = pd.read_csv("../../data/preprocessed/base.csv", sep=',')
df.shape

(2945818, 13)

In [4]:
#### COMBINATORIA ####
data = dataset.combinatoria_periodo_producto()
data['periodo'] = data['periodo'].dt.year * 100 + data['periodo'].dt.month
data.shape

(44388, 2)

In [5]:
productos_ok = pd.read_csv("../../data/raw/product_id_apredecir201912.csv", sep="\t")
data = data[data['product_id'].isin(productos_ok['product_id'].unique())]
data

,product_id,periodo
0,20524,201701
1,20524,201702
2,20524,201703
3,20524,201704
4,20524,201705
...,...,...
42835,20127,201908
42836,20127,201909
42837,20127,201910
42838,20127,201911


In [6]:
#### MERGE CON PRODUCTOS ####
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
data = data.merge(productos, how='left', on="product_id")
del productos

#### MERGE CON STOCKS ####
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')
stocks = stocks.groupby(by=["periodo", "product_id"]).agg({"stock_final": "sum"}).reset_index()
data = data.merge(stocks, how='left', on=['periodo', 'product_id'])
del stocks

#### MERGE CON SELLIN ####
sellin = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
sellin = sellin.groupby(by=["periodo","product_id"]).agg({"tn":"sum", "plan_precios_cuidados":"sum", "cust_request_qty":"sum", "cust_request_tn":"sum"}).reset_index()
data = data.merge(sellin, how='left', on=['periodo', 'product_id'])
del sellin
gc.collect()

20

In [8]:
#### COMPLETO TN CON CEROS ####
####  ¿cuantos?
print(f"Total de periodos con Nan debido a la combinatoria periodo_x_producto: {data['tn'].isna().sum()}")
#### Lo completo con ceros
data['tn'] = data['tn'].fillna(0)

Total de periodos con Nan debido a la combinatoria periodo_x_producto: 5731


In [9]:
#### GUARDAR DATAFRAME ####
data.to_csv("./datasets/periodo_x_producto.csv", index=False, sep=',', encoding='utf-8')

In [10]:
#### GUARDAR DATAFRAME ####
data = pd.read_csv("./datasets/periodo_x_producto.csv", sep=',')

In [ ]:
import pandas as pd
from prophet import Prophet
from tqdm import tqdm

# ---------------------
# 🛠 Preparación inicial
# ---------------------

# Creo DF
df = data[['product_id','periodo', 'tn']].copy()

# Convertir 'periodo' (yyyymm) a datetime
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
df = df.sort_values(['product_id', 'periodo_dt'])  # Ordenar por producto y fecha

df.drop(columns=['periodo'], inplace=True)  # Eliminar columna temporal
df.rename(columns={'tn': 'y', 'periodo_dt':'ds'}, inplace=True)  # Renombrar columna de tn

productos_ok = pd.read_csv("../../data/raw/product_id_apredecir201912.csv", sep="\t")



# Para guardar las predicciones
resultados = []

# ---------------------
#  Loop por producto
# ---------------------
for product_id in productos_ok['product_id'].unique():
    df_prod = df[df['product_id'] == product_id].sort_values('ds')

    if len(df_prod) < 6:
        continue  # opcional: saltear series demasiado cortas

    # Entrenar Prophet
    model = Prophet(yearly_seasonality=True)
    model.fit(df_prod[['ds', 'y']])

    # Predecir mes +2
    ultima_fecha = df_prod['ds'].max()
    future = model.make_future_dataframe(periods=2, freq='MS')
    future = future[future['ds'] > ultima_fecha]  # solo fechas futuras
    pred = model.predict(future)

    # Obtener solo la predicción de mes+2
    pred_mes2 = pred.tail(1)

    resultados.append({
        'product_id': product_id,
        'fecha_predicha': pred_mes2['ds'].values[0],
        'yhat': pred_mes2['yhat'].values[0],
        'yhat_lower': pred_mes2['yhat_lower'].values[0],
        'yhat_upper': pred_mes2['yhat_upper'].values[0]
    })

# ---------------------
# 📊 Resultados finales
# ---------------------
df_predicciones = pd.DataFrame(resultados)
print(df_predicciones.head())


In [ ]:
df_predicciones.rename(columns={'yhat': 'tn'}, inplace=True)
df_predicciones[['product_id', 'tn']].to_csv("prophet_v2.csv", index=False, sep=',', encoding='utf-8')

In [4]:
df_kaggle = pd.read_csv("./datasets/prophet_v2.csv", sep=',')
df_kaggle.rename(columns={'yhat': 'tn'}, inplace=True)
df_kaggle[['product_id', 'tn']].to_csv("./datasets/prophet_v2_kaggle.csv", index=False, sep=',', encoding='utf-8')

In [ ]:
df_kaggle = pd.read_csv("./datasets/prophet_v2.csv", sep=',')
df_kaggle.rename(columns={'yhat_lower': 'tn'}, inplace=True)
df_kaggle[['product_id', 'tn']].to_csv("./datasets/prophet_yhat_lower.csv", index=False, sep=',', encoding='utf-8')